In [7]:
import requests      
import numpy as np   
import pandas as pd  
import time          

from bs4 import BeautifulSoup
import re

In [8]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36'

In [9]:
# !/usr/local/anaconda3/bin/pip install fake_useragent

In [10]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [11]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

Current IP Address: 185.220.101.70


In [14]:
MAIN_LINK = "http://k.intimcity.nl/persons.php?index={}"

In [15]:
page_link = "http://k.intimcity.nl/persons.php?id=352916"

PERSON_LINK = "http://k.intimcity.nl/persons.php?id={}"

In [9]:
main_link.format(1)

'http://k.intimcity.nl/persons.php?index=1'

In [16]:
def get_soup(link):
    headers = {'User-Agent': UserAgent().chrome}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [17]:
def get_main_page(page_number, main_link=MAIN_LINK):
    main_page = get_soup(main_link.format(page_number))
    return main_page

In [18]:
def get_max_main_page(main_page):
    table = main_page.find('table', attrs={'style':'margin: auto; border-spacing: 3px;'})
    max_page = int(table.findAll('a', attrs={'class':'index'})[-2].text)
    return max_page

In [21]:
def get_persons_links(current_main_page, person_link=PERSON_LINK):
    persons_links = []
    page_ids = current_main_page.findAll('a', attrs={'class':'x'})
    for page_id in page_ids:
        page_id = re.findall(r'\d+', page_id.get('onclick'))[0]
        link = person_link.format(page_id)
        persons_links.append(link)
    return persons_links

In [22]:
current_main_page = get_main_page(1)
current_persons_links = get_persons_links(current_main_page)

In [23]:
len(current_persons_links)

50

In [24]:
len(set(current_persons_links))

50

In [36]:
current_persons_links[0]

'http://k.intimcity.nl/persons.php?id=374085'

In [221]:
current_person_page = get_soup('http://k.intimcity.nl/persons.php?id=371614')

In [28]:
def get_text_description(current_person_page):
    return {'text_description': current_person_page.find('td', attrs={'class':'ar13'}).text}

In [29]:
def get_parameter_table(current_person_page):
    parameter_table = current_person_page.find("table", attrs={'style':'width: 100%; border-spacing: 1px;'})
    parameter_table = pd.read_html(str(parameter_table))[0]
    parameter_table = parameter_table.set_index(0)
    parameter_table = parameter_table.to_dict()[1]
    return parameter_table

In [152]:
def get_prices_dict(current_person_page, flatten=True):
    
    def _flatten_prices_dict(prices_dict):
        flattened_prices = {}
        for k, v in prices_dict.items():
            if isinstance(v, dict):
                for nested_k, nested_v in v.items():
                    if isinstance(nested_v, dict):
                        for nested_2_k, nested_2_v in nested_v.items():
                            flattened_prices["_".join([k, nested_k, nested_2_k])] = nested_2_v
                    else:
                        flattened_prices["_".join([k, nested_k])] = nested_v
            else:
                flattened_prices[k] = v
        return flattened_prices
    
    def _clean_number(df, row_index, col_index):
        try:
            string = df.iloc[row_index, col_index]
        except:
            return None
        try:
            return float("".join(re.findall(r'\d+', string)))
        except:
            return string
           
    prices = current_person_page.find(
        'table', 
        attrs={
            'style':'width: 440px; margin: 10px auto; background-color: #CCC; text-align: center; border-spacing: 1px;'
        }
    )
    prices = pd.read_html(str(prices))[0]
        
    prices_dict = {
        "апартаменты":{
            "днём": {
                "час": _clean_number(prices, 2, 1),
                "2часа": _clean_number(prices, 2, 2)
            },
            "ночью": {
                "час": _clean_number(prices, 2, 3),
                "2часа": _clean_number(prices, 2, 4)
            }
        },
        "выезд":{
            "днём": {
                "час": _clean_number(prices, 3, 1),
                "2часа": _clean_number(prices, 3, 2)
            },
            "ночью": {
                "час": _clean_number(prices, 3, 3),
                "2часа": _clean_number(prices, 3, 4)
            }
        },
        "контактов_в_час": _clean_number(prices, 4, 1),
        "анал": _clean_number(prices, 4, 4),
        "мбр": _clean_number(prices, 5, 4),
        "comments": _clean_number(prices, 6, 0)
    }
    
    if flatten:
        prices_dict = _flatten_prices_dict(prices_dict)
        
    return prices_dict

In [201]:
def get_services_dict(current_person_page):

    services = current_person_page.find("table", attrs={'style':'width: 100%; border-spacing: 5px;'})

    # boolean mask for cheked boxes
    services_bool = services.findAll('input', attrs={'type':'checkbox'})[:-1]
    services_bool = [1 if 'checked' in str(x) else 0 for x in services_bool]
    
    services_names = services.findAll('a')
    services_names = [name.text for name in services_names]
    
    services_dict = dict(zip(services_names, services_bool))
    return services_dict

In [216]:
def get_photo_links(current_person_page):
    photo_links = current_person_page.find("div", attrs={'class':'highslide-gallery'})
    photo_links = photo_links.findAll('a')
    photo_links = [link.get("href") for link in photo_links]
    return photo_links

In [322]:
def get_checks_reviews(current_person_page):
    
    comments = current_person_page.find('div', attrs={'class':'noprint'}) 
    
    number_of_checks = comments.find('table', attrs={'style':'margin: auto;'})
    number_of_checks = len(number_of_checks.findAll('div', attrs={'class':'rnd5'})) if number_of_checks else 0
    # reviews from users
    reviews = comments.find('table', attrs={'style':'width: 440px; margin: 10px auto 0 auto;'})    
    reviews = reviews.findAll('div', attrs={'class':'rnd5'}) if reviews else []

    clean_reviews = []
    for review in reviews:
        span_tag = review.find('span')
        review_text = span_tag.next_sibling
        try:
            clean_text = review_text.text
        except:
            clean_text = review_text
        clean_reviews.append(clean_text)
        
    number_of_reviews = len(clean_reviews)
    
    result = {
        'number_of_checks':number_of_checks,
        'number_of_reviews':number_of_reviews,
        'reviews_text':clean_reviews
    }
    return result

In [323]:
parameter_table = get_parameter_table(current_person_page)

In [324]:
text_description = get_text_description(current_person_page)

In [325]:
prices = get_prices_dict(current_person_page)

In [326]:
services = get_services_dict(current_person_page)

In [327]:
photo_links = get_photo_links(current_person_page)

In [328]:
checks_reviews = get_checks_reviews(current_person_page)